In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import folium
import requests
import logging
from folium.plugins import TimestampedGeoJson, HeatMap

pd.options.display.max_colwidth = 200

In [2]:
file_name = 'simulation_output_example.csv'

data = pd.read_csv(file_name, sep=';',
                   # convert column to dict
                   converters={'details': lambda v: eval(v)})

print(f'Data shape: {data.shape}')

data.head()

Data shape: (726, 9)


,clock_time,object_type,uuid,itinerary_id,from_state,to_state,lon,lat,details
0,0,vehicle,b59174a7c5634908951530d2df4a0315,NaN,offline,idling,-73.997155,40.711960,{}
1,0,vehicle,16e94eeb59824f768415a430413353ef,NaN,offline,idling,-73.982318,40.769531,{}
2,0,vehicle,5cd0f7844fb2413bb4a539cbb55ea08f,NaN,offline,idling,-73.982430,40.785004,{}
3,0,vehicle,277a72c303bb47d1a33d0c82bf3bf5a8,NaN,offline,idling,-74.007424,40.726010,{}
4,0,vehicle,bac82b689bea41e78a0e78a1e43c55db,NaN,offline,idling,-73.978013,40.758114,{}


In [3]:
# convert "details" column into a separate DataFrame
details = data.details.apply(pd.Series)
details[~data.itinerary_id.isna()].head()

,dropoff,origin,destination,trip_distance,trip_duration,pickup,stop
11,NaN,"{'lat': 40.758114, 'lon': -73.978013}","{'lat': 40.752995, 'lon': -73.975136}",0.0,0.0,56f96379691e416f87dfb57f687379cc,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,"{'lat': 40.744255, 'lon': -73.973064}","{'lat': 40.751465, 'lon': -73.976341}",0.0,0.0,530f8908495149a686bffe4497b5d4ff,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pickup locations

In [4]:
idx = (data.object_type == 'booking') & (data.from_state == 'pickup')
points = data[idx][['lat', 'lon']]

m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

HeatMap(points, radius=15, min_opacity=0.5).add_to(m)

m

### Vehicle movements

**WARNING**: requires [OSRM]('http://localhost:5010/route/v1/driving/-73.977547,40.763336;-73.938263,40.791920?geometries=geojson') running localy on port 5010. 

In [5]:
from requests.exceptions import ConnectionError
    
OSRM_HOST = 'localhost'
OSRM_PORT = 5010
OSRM_URL_FORMAT = 'http://{osrm_host}:{osrm_port}/route/v1/driving/{coords}?geometries=geojson'

def get_vehicle_route(from_point, to_point):
    """lon/lat order"""
    
    geoms = []
    
    try:
        coords = f'{from_point[0]},{from_point[1]};{to_point[0]},{to_point[1]}'
        url = OSRM_URL_FORMAT.format(
            osrm_host=OSRM_HOST, 
            osrm_port=OSRM_PORT, 
            coords=coords)

        # TODO: check response code
        osrm_response = requests.get(url)
        
        for route in osrm_response.json()['routes']:
            geom = {
                'type': 'Feature',
                'geometry': route['geometry'],
                'properties': {}
            }
            
            geoms.append(geom)
            
        return geoms
    except ConnectionError as e:
        logging.warning(e)
        
        geom = {
            'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates': [list(from_point), list(to_point)]
            },
            'properties': {}
        }
    return geoms

In [6]:
m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

def style_function(feature):
    colors = ['red', 'blue', 'brown', 'yellow', 'green']
    idx = feature['properties']['vehicle_index']
    if idx >= len(colors):
        idx = -1
    
    return {
        'opacity': 0.5, 
        'weight': 3,
        'color': colors[idx]
    }

num_vehicles = 0
total_vehicles = 3

all_routes = []
now = datetime.now()

idx = (data.object_type == 'vehicle')

for i, (grp, vehicle_changes) in enumerate(data[idx].groupby('uuid')):
    for grp, items in vehicle_changes.groupby('itinerary_id'):
        line = []
        time_steps = []
        for item in items.itertuples():
            line.append([item.lon, item.lat])
            
            tm = now + timedelta(minutes=(item.clock_time))
            time_steps.append(tm)

        # only 2 points - from and to
        if len(line) != 4:
            continue
            
        for from_point, to_point in zip(line[::2], line[1::2]):
            
            for geom in get_vehicle_route(from_point, to_point):
                start_at = time_steps.pop(0)
                stop_at = time_steps.pop(0)

                num_points = len(geom['geometry']['coordinates'])
                geom['properties']['times'] = list(pd.date_range(start_at, stop_at, num_points).astype(str))
                geom['properties']['vehicle_index'] = i
                geom['properties']['style'] = style_function(geom)
                
                all_routes.append(geom)
                
                folium.GeoJson(
                    geom,
                    name='geojson',
                    style_function=style_function
                ).add_to(m)
                
    num_vehicles += 1
    if num_vehicles == total_vehicles:
        break
            
m

### Time stamped moves

In [7]:
m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

TimestampedGeoJson(
    all_routes,
    period='PT10M',
    add_last_point=False,
    auto_play=False, 
    loop=False, 
    max_speed=1,
    loop_button=True,
    time_slider_drag_update=True
).add_to(m)

m